# Val and Marta's Final Project
## CSE360/460

### Frisbee-Catching Robot

In [1]:
import sim
%pylab inline
user_ip = '192.168.32.1' #'127.0.0.1'

Populating the interactive namespace from numpy and matplotlib


In [2]:
class robot():
    
    def __init__(self, frame_name, motor_names=[], client_id=0):  
        # If there is an existing connection
        if client_id:
                self.client_id = client_id
        else:
            self.client_id = self.open_connection()
            
        self.motors = self._get_handlers(motor_names) 
        
        # Robot frame
        self.frame =  self._get_handler(frame_name)
            
        
    def open_connection(self):
        sim.simxFinish(-1)  # just in case, close all opened connections
        self.client_id = sim.simxStart(user_ip, 19999, True, True, 5000, 5)  # Connect to CoppeliaSim 
        
        if clientID != -1:
            print('Robot connected')
        else:
            print('Connection failed')
        return clientID
        
    def close_connection(self):    
        sim.simxGetPingTime(self.client_id)  # Before closing the connection to CoppeliaSim, make sure that the last command sent out had time to arrive.
        sim.simxFinish(self.client_id)  # Now close the connection to CoppeliaSim:
        print('Connection closed')
    
    def isConnected(self):
        c,result = sim.simxGetPingTime(self.client_id)
        # Return true if the robot is connected
        return result > 0         
        
    def _get_handler(self, name):
        err_code, handler = sim.simxGetObjectHandle(self.client_id, name, sim.simx_opmode_blocking)
        return handler
    
    def _get_handlers(self, names):
        handlers = []
        for name in names:
            handler = self._get_handler(name)
            handlers.append(handler)
        
        return handlers

    def send_motor_velocities(self, vels):
        for motor, vel in zip(self.motors, vels):
            err_code = sim.simxSetJointTargetVelocity(self.client_id, 
                                                      motor, vel, sim.simx_opmode_streaming)      
            
    def set_position(self, position, relative_object=-1):
        if relative_object != -1:
            relative_object = self._get_handler(relative_object)        
        sim.simxSetObjectPosition(clientID, self.frame, relative_object, position, sim.simx_opmode_oneshot)
        
    def simtime(self):
        return sim.simxGetLastCmdTime(self.client_id)
    
    def get_position(self, relative_object=-1):
        # Get position relative to an object, -1 for global frame
        if relative_object != -1:
            relative_object = self._get_handler(relative_object)
        res, position = sim.simxGetObjectPosition(self.client_id, self.frame, relative_object, sim.simx_opmode_blocking)        
        return array(position)
    
    def get_object_position(self, object_name):
        # Get Object position in the world frame
        err_code, object_h = sim.simxGetObjectHandle(self.client_id, object_name, sim.simx_opmode_blocking)
        res, position = sim.simxGetObjectPosition(self.client_id, object_h, -1, sim.simx_opmode_blocking)
        return array(position)
    
    def get_object_relative_position(self, object_name):        
        # Get Object position in the robot frame
        err_code, object_h = sim.simxGetObjectHandle(self.client_id, object_name, sim.simx_opmode_blocking)
        res, position = sim.simxGetObjectPosition(self.client_id, object_h, self.frame, sim.simx_opmode_blocking)
        return array(position)

In [19]:
def point_to_point_traj(x1, x2, v1, v2, delta_t):
    t = np.linspace(0, delta_t, 100)  
    a0 = x1
    a1 = v1
    a2 = (3*x2 - 3*x1 - 2*v1*delta_t - v2 * delta_t) / (delta_t**2)
    a3 = (2*x1 + (v1 + v2) * delta_t  - 2 * x2) / (delta_t**3)

    polynomial = a0 + a1 * t + a2 * t**2 + a3 * t**3
    derivative = a1 + 2*a2 * t + 3 * a3 * t**2
    #print('a0: (%.4f) '%(a0))
    #print('a1: (%.4f) '%(a1))
    #print('a2: (%.4f) '%(a2))
    #print('a3: (%.4f) '%(a3))
    return polynomial, derivative

def a_pairs(x1, x2, v1, v2, delta_t):
    t = np.linspace(0, delta_t, 100)  
    a0 = x1
    a1 = v1
    a2 = (3*x2 - 3*x1 - 2*v1*delta_t - v2 * delta_t) / (delta_t**2)
    a3 = (2*x1 + (v1 + v2) * delta_t  - 2 * x2) / (delta_t**3)

    #print('a0: (%.4f) '%(a0))
    #print('a1: (%.4f) '%(a1))
    #print('a2: (%.4f) '%(a2))
    #print('a3: (%.4f) '%(a3))
    return a0, a1, a2, a3

def piecewise3D (X,Y,Z, Vx, Vy, Vz, T):
    theta_x, theta_y, theta_z, dx, dy, dz = [], [], [], [], [], []
    a0_pairs = []
    a1_pairs = []
    a2_pairs = []
    a3_pairs = []

    
    count = 1
    for i in range(len(P)-1):
       # print(count)
        count = count+1
       # print('x')
        theta_xi, dxi = point_to_point_traj(X[i], X[i+1], Vx[i], Vx[i+1], T[i+1] - T[i])
        xa0, xa1, xa2, xa3 = a_pairs(X[i], X[i+1], Vx[i], Vx[i+1], T[i+1] - T[i])
       # print('y')
        theta_yi, dyi = point_to_point_traj(Y[i], Y[i+1], Vy[i], Vy[i+1], T[i+1] - T[i])
        ya0, ya1, ya2, ya3 = a_pairs(Y[i], Y[i+1], Vy[i], Vy[i+1], T[i+1] - T[i])
       # print('z')
        theta_zi, dzi = point_to_point_traj(Z[i], Z[i+1], Vz[i], Vz[i+1], T[i+1] - T[i])
        za0, za1, za2, za3 = a_pairs(Z[i], Z[i+1], Vz[i], Vz[i+1], T[i+1] - T[i])
       # print('=================')
        
        a0_pairs.append([xa0, ya0, za0])
        a1_pairs.append([xa1, ya1, za1])
        a2_pairs.append([xa2, ya2, za2])
        a3_pairs.append([xa3, ya3, za3])
        
        theta_x += theta_xi.tolist()
        theta_y += theta_yi.tolist()
        theta_z += theta_zi.tolist()
        dx += dxi.tolist()
        dy += dyi.tolist()
        dz += dzi.tolist()

        #plot(theta_xi, theta_yi)
    return theta_x, theta_y, theta_z, dx, dy, dz, a0_pairs, a1_pairs, a2_pairs, a3_pairs

# Plotting
def plot_points():
    plot(X,Y, '--')
    plot(X,Y, 'o')
    quiver(X,Y, Vx, Vy, color='r')

def plot_points_3d():
    plot(X,Y,Z, '--')
    plot(X,Y,Z, 'o')
    
# Speed
def plot_speed():
    speed = np.sqrt(np.array(dx)**2 + np.array(dy)**2)
    plot(speed)

In [4]:
import sim as vrep
import time
import cv2
import numpy as np
from matplotlib.transforms import Affine2D
from pathfinding.core.diagonal_movement import DiagonalMovement
from pathfinding.core.grid import Grid
from pathfinding.finder.a_star import AStarFinder

# Tha map is 20x20 sq meters
#Lets define a grid of nxn
n = 20
#gmap = zeros(n*n) # the map is a grid of nxn

# x and y coordinates for the grid cells. Lowest and leftest point in the cell.
cell_w = 20/n
grid_x, grid_y = np.mgrid[-10:10:cell_w,-10:10:cell_w]
#print(len(grid_x))
# Convert the matrix into a vector
grid_x = grid_x.flatten()
grid_y = grid_y.flatten()

#plot(grid_x, grid_y, '+')

##get trees
tree_names = ["Tree","Tree#1","Tree#2","Tree#3","Tree#5"]
#locations = []
xzcs = []
vrep.simxFinish(-1)
#connect to the server 
clientID = vrep.simxStart(user_ip, 19999, True, True, 5000, 5)

if clientID != -1:
    print('Server is connected!')
else:
    print('Server is unreachable!')
    sys.exit(0)

for name in tree_names:
    errorcode, handle = vrep.simxGetObjectHandle(clientID,name,vrep.simx_opmode_blocking)
    errorcode, location = vrep.simxGetObjectPosition(clientID,handle,-1,vrep.simx_opmode_blocking)
    #plot(location[0],location[1],'*')
    #locations.append([round(location[0]), round(location[1])])
    loc = np.array((round(location[0]), round(location[1])))
    xc = loc[0]
    zc = loc[1]
    xzcs.append([xc,zc])
    
#print(locations)
#populate_trees(locations[0],n)
print(xzcs)

def points_cell(x, y, d):    
    X = [x, x+d, x+d, x]
    Y = [y, y, y+d, y+d]
    return X, Y

#fig = plt.figure()
#ax = fig.add_subplot(111, aspect='equal')
# plot each cell
for x, y in zip(grid_x, grid_y):
    X, Y = points_cell(x, y, cell_w)
    #cell = plt.Polygon([(xi, yi) for xi, yi in zip(X,Y)], color='0.9')
    
    #ax.add_patch(cell)

    #plot(X,Y, 'k-')
    #plot(X,Y, 'b+')    
    
#fig.canvas.draw()

l0 = (0.3/(1-0.3))  # Initial belief
gmap = l0 * ones(n*n) # Initial belief
# For each cell, check if the circle is in it.
count = 0
ret_locs = []
for i in range(n*n):
    x, y = grid_x[i], grid_y[i]
    #TODO Run this for each sphere (center and radius)
    # Corners of the cell
    X, Y = points_cell(x, y, cell_w)
    # check based on the ecuclidean distance
    for j in range(0,len(xzcs)):
        xc,zc = xzcs[j]
        dist = sqrt((xc - X)**2 + (zc - Y)**2)
        radius = 2
        k = radius / (1.5 / 2)
        # Check if At least one of the borders is within the sphere
        if((dist < radius/k).any()):
            #print(dist)
            po = 0.8  # P(mi/zt) probability of having an obstacle 
            li = log(po / (1-po)) + gmap[i] - l0
            gmap[i] = li  # P(mi/zt) 
            #print(li)
            ret_locs.append([x,y])
        else:
            po = 0.05  # P(mi/zt) probability of having an obstacle given a non-detected obstacle

            # Cells within the fov. Check if the four points are withing the FOV
            #thetas = np.arctan2(Y,X) - pi/2 - angle
            thetas = 2*pi
            if np.logical_and(-pi/6 <thetas, thetas < pi/6).all():
                li = log(po / (1-po)) + gmap[i] - l0
                gmap[i] = li

            pass
print(ret_locs)
#gmap = nan_to_num(gmap)
# normalize gmap
gmap = gmap - min(gmap)
gmap = gmap / max(gmap)


Server is connected!
[[-1, -6], [3, 4], [-4, 5], [-4, -1], [4, -1]]
[[-5.0, -2.0], [-5.0, -1.0], [-5.0, 4.0], [-5.0, 5.0], [-4.0, -2.0], [-4.0, -1.0], [-4.0, 4.0], [-4.0, 5.0], [-2.0, -7.0], [-2.0, -6.0], [-1.0, -7.0], [-1.0, -6.0], [2.0, 3.0], [2.0, 4.0], [3.0, -2.0], [3.0, -1.0], [3.0, 3.0], [3.0, 4.0], [4.0, -2.0], [4.0, -1.0]]


In [22]:
def get_pairs_for_traj(start,end):
    n, m = 20,20  # number of rows and columns respectively.
    # to convert to coppeliasim, the origin of the graph needs to be moved up and over 10 by 10
    # Create a matrix to represent the cells of the grid
    grid_cells = np.ones((n,m))
    for loc in ret_locs:
        #print(int(loc[0]+10))
        grid_cells[int(loc[0]+10)][int(loc[1]+10)] = 0;

    grid = Grid(matrix=grid_cells)
    #Get robot current position
    start = grid.node(3, 1)
    #Get robot end position
    end = grid.node(17, 18)
    finder = AStarFinder(diagonal_movement=DiagonalMovement.always)
    path, runs = finder.find_path(start, end, grid)
    #print(path)
    c_path = [];
    for coord in path:
        x = 10-coord[0]
        y = 10-coord[1]
        z = 1 #basic height
        c_path.append((x,y,z))

    # Velocities
    vs = [[0,0,0]]
    for c_index in range(0,len(c_path)-1):
        rn = c_path[c_index]
        fut = c_path[c_index+1]
        xs = fut[0]-rn[0]
        ys = fut[1]-rn[1]
        zs = fut[2]-rn[2]
        if xs != 0:
            xs = xs/8
        if ys != 0:
            ys = ys/8
        if zs != 0:
            zs = zs/8
        vs.append([xs,ys,zs])
    #print(vs)

    # Time
    ts = []
    for tim in range(0,len(c_path)):
        ts.append(tim*10)

    # Grouping
    #P = np.vstack((p1, p2, p3, p4, p5, p6, p7, p8, p9, p10, p11, p12, p13, p14, p15, p16, p17))
    #V = np.vstack((v1, v2, v3, v4, v5, v6, v7, v8, v9, v10, v11, v12, v13, v14, v15, v16, v17))
    #T = [t1, t2, t3, t4, t5, t6, t7, t8, t9, t10, t11, t12, t13, t14, t15, t16, t17]
    P = np.vstack(c_path)
    V = np.vstack(vs)
    T = ts
    X, Y, Z = P[:,0], P[:,1], P[:,2]
    Vx, Vy, Vz = V[:,0], V[:,1], V[:,2]

    # Plotting
    #plot_points()
    #show()

    # Piecewise function
    theta_x, theta_y, theta_z, dx, dy, dz, a0_pairs, a1_pairs, a2_pairs, a3_pairs = piecewise3D(X,Y,Z, Vx,Vy,Vz, T)
    return a0_pairs, a1_pairs, a2_pairs, a3_pairs